In [ ]:
import os
import conda
import pandas as pd
import numpy as np
import json
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

In [ ]:
# %matplotlib inline

In [ ]:
HUC8  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
# HUC8.crs = {'init' :'epsg:4326'}
HUC8['HUC_8']=HUC8['HUC_8'].astype(int)
HUC8

In [ ]:
#Ecoregions
Ecoregions  = geopandas.GeoDataFrame.from_file('NARS_NP_values/narswsa_20110504.shp')
Ecoregions.crs = {'init' :'epsg:4326'}

In [ ]:
# For folium
HUC8_json = HUC8.to_crs(epsg='4326').to_json()

In [ ]:
import folium

m = folium.Map(location=[48, -102], zoom_start=3)

folium.GeoJson(
    HUC8_json,
    name='HUC8'
).add_to(m)
# m
m.save('HUC8.html')


In [ ]:
# proj = gcrs.AlbersEqualArea()
# gplt.polyplot(HUC8, projection=proj, edgecolor='white', linewidth=0.1, facecolor='black',)

In [ ]:
chla_df = pd.read_csv('chla/nla2012_chla_wide.csv', sep=",", header=0)
chla_df

In [ ]:
NLA_site_info = pd.read_csv('NLA_site_info/nla2012_wide_siteinfo_08232016.csv', sep=",", header=0)
NLA_site_info

In [ ]:
# =============================================================================
# NLA (lakes) UID:HUC8 correspondence
# =============================================================================
    
# UID_HUC8_dict
UID_HUC8_dict_NLA = {key: NLA_site_info['HUC8'][NLA_site_info.UID == key].values for key in NLA_site_info['UID'].values}
UID_LAT_DD83_dict_NLA = {key: NLA_site_info['LAT_DD83'][NLA_site_info.UID == key].values for key in NLA_site_info['UID'].values}
UID_LON_DD83_dict_NLA = {key: NLA_site_info['LON_DD83'][NLA_site_info.UID == key].values for key in NLA_site_info['UID'].values}
UID_AGGR_ECO9_2015_dict_NLA = {key: NLA_site_info['AGGR_ECO9_2015'][NLA_site_info.UID == key].values for key in NLA_site_info['UID'].values}
UID_SITE_ID_2015_dict_NLA = {key:NLA_site_info['SITE_ID'][NLA_site_info.UID == key].values for key in NLA_site_info['UID'].values}

aux_list_HUC8_NLA = []
aux_list_LAT_DD83_NLA = []
aux_list_LON_DD83_NLA = []
aux_list_AGGR_ECO9_2015_NLA = []
aux_list_SITE_ID_2015_NLA = []
for i in chla_df['UID']:
    aux_list_HUC8_NLA.append(int(UID_HUC8_dict_NLA[i]))
    aux_list_LAT_DD83_NLA.append(float(UID_LAT_DD83_dict_NLA[i]))
    aux_list_LON_DD83_NLA.append(float(UID_LON_DD83_dict_NLA[i]))
    aux_list_AGGR_ECO9_2015_NLA.append(str(UID_AGGR_ECO9_2015_dict_NLA[i]))
    aux_list_SITE_ID_2015_NLA.append(str(UID_SITE_ID_2015_dict_NLA[i]))

chla_df['HUC8'] = aux_list_HUC8_NLA
chla_df['LAT_DD83'] = aux_list_LAT_DD83_NLA
chla_df['LON_DD83'] = aux_list_LON_DD83_NLA
chla_df['AGGR_ECO9_2015'] = aux_list_AGGR_ECO9_2015_NLA
chla_df['SITE_ID'] = aux_list_SITE_ID_2015_NLA
chla_df[['UID','HUC8','LAT_DD83','LON_DD83', 'AGGR_ECO9_2015','SITE_ID']]

# NLA_points = NLA_points.columns.get_level_values(0)
chla_df.columns = chla_df.columns.to_flat_index()
chla_df = chla_df.rename(columns={'HUC8':'HUC_8'})
chla_df

In [ ]:
chla_df_filt = chla_df.dropna(subset=['CHLX_RESULT']) #remove NAN values of CHLX_RESULT


In [ ]:
chla_df_group_SITE_ID_fil = chla_df_filt.groupby(['SITE_ID']).mean()
chla_df_group_SITE_ID_fil

In [ ]:
chla_df_group_SITE_ID_HUC_8_filt = chla_df_group_SITE_ID_fil.groupby(['HUC_8']).mean()
chla_df_group_SITE_ID_HUC_8_filt

In [ ]:
# Merge dataframe and geodataframe
HUC8_chla = HUC8.merge(chla_df_group_SITE_ID_HUC_8_filt, on='HUC_8')
HUC8_chla

In [ ]:
# # DEPRECATED IN FAVOR OF ZEROS

# # =============================================================================
#     # Fill the gaps with the ecoregions average data
#     # =============================================================================
# HUC8_in_chla_list = list(HUC8_chla['HUC_8'])
# HUC8_total_list = list(HUC8['HUC_8'])
# HUC8_diff_list = set(HUC8_total_list)-set(HUC8_chla)

# HUC8_diff = HUC8[HUC8['HUC_8'].isin(HUC8_diff_list)]

# HUC8inEcoregions = sjoin(HUC8_diff, Ecoregions, how='inner', op='intersects')
# HUC8inEcoregions = HUC8inEcoregions.drop_duplicates(subset=['HUC_8'])
# HUC8inEcoregions_index = HUC8inEcoregions.set_index('HUC_8')

# HUC8_Ecoregion_dict = dict(zip(HUC8inEcoregions.HUC_8, HUC8inEcoregions.WSA_REGION))

# NARS_NLA_AGGR_ECO9_2015 = chla_df.groupby(['AGGR_ECO9_2015']).mean()
# NARS_NLA_AGGR_ECO9_2015 = NARS_NLA_AGGR_ECO9_2015.rename(index={"['CPL']":'CPL',
#                                      "['NAP']":'NAP',
#                                      "['NPL']":'NPL',
#                                      "['SAP']":'SAP',
#                                      "['SPL']":'SPL',
#                                      "['TPL']":'TPL',
#                                      "['UMW']":'UMW',
#                                      "['WMT']":'WMT',
#                                      "['XER']":'XER',
#                                     })

# HUC8_Ecoregion_chla_dict = dict()
# for l in HUC8_diff.HUC_8:
#     HUC8_in_chla_list = list(HUC8_chla['HUC_8'])
# HUC8_total_list = list(HUC8['HUC_8'])
# HUC8_diff_list = set(HUC8_total_list)-set(HUC8_in_chla_list)

# HUC8_diff = HUC8[HUC8['HUC_8'].isin(HUC8_diff_list)]

# HUC8inEcoregions = sjoin(HUC8_diff, Ecoregions, how='inner', op='intersects')
# HUC8inEcoregions = HUC8inEcoregions.drop_duplicates(subset=['HUC_8'])
# HUC8inEcoregions_index = HUC8inEcoregions.set_index('HUC_8')

# HUC8_Ecoregion_dict = dict(zip(HUC8inEcoregions.HUC_8, HUC8inEcoregions.WSA_REGION))

# NLA_AGGR_ECO9_2015 = chla_df.groupby(['AGGR_ECO9_2015']).mean()
# NLA_AGGR_ECO9_2015 = NLA_AGGR_ECO9_2015.rename(index={"['CPL']":'CPL',
#                                      "['NAP']":'NAP',
#                                      "['NPL']":'NPL',
#                                      "['SAP']":'SAP',
#                                      "['SPL']":'SPL',
#                                      "['TPL']":'TPL',
#                                      "['UMW']":'UMW',
#                                      "['WMT']":'WMT',
#                                      "['XER']":'XER',
#                                     })

# HUC8_Ecoregion_chla_dict = dict()
# for l in HUC8_diff.HUC_8:
#     HUC8_Ecoregion_chla_dict[l] = NLA_AGGR_ECO9_2015.loc[HUC8_Ecoregion_dict[l],'CHLX_RESULT']



In [ ]:
#Zeros
HUC8_Ecoregion_chla= pd.DataFrame(HUC8_diff.HUC_8, columns=['HUC_8'])
HUC8_Ecoregion_chla['CHLX_RESULT'] = np.nan

HUC8_Ecoregion_chla = HUC8_Ecoregion_chla.set_index('HUC_8', drop=True)

NARS_NLA_FINAL_chla = pd.concat([chla_df_group_SITE_ID_HUC_8_filt, HUC8_Ecoregion_chla], ignore_index=False, sort=False)

HUC8_NARS_NLA_FINAL_chla = HUC8.merge(NARS_NLA_FINAL_chla, on='HUC_8')
# HUC8_NARS_NLA_FINAL_chla['CHLX_RESULT'] = HUC8_NARS_NLA_FINAL_chla['CHLX_RESULT'].replace(0, np.nan, inplace=True)

In [ ]:
# proj = gcrs.AlbersEqualArea()
# norm = colors.LogNorm()
# cmap = matplotlib.cm.viridis
# cmap.set_under('grey')
# ax = gplt.choropleth(HUC8_NARS_NLA_FINAL_chla, hue='CHLX_RESULT', projection=proj, norm=norm, cmap=cmap, k=7, scheme=None, legend=True, legend_kwargs={'loc': 'lower right'},figsize=(12, 12))#, vmin=HUC8_NARS_NLA_FINAL_chla[HUC8_NARS_NLA_FINAL_chla['CHLX_RESULT'] > 0]['CHLX_RESULT'].min())#, vmin=0.8, vmax=HUC8_NARS_NLA_FINAL['PTL'].max() , linewidth=0.5, edgecolor='black',)
# # gplt.pointplot(point, ax=ax, projection=proj, s=300, color='none', edgecolors='r', linewidth='3')
# # gplt.pointplot(point, ax=ax, projection=proj, color= 'red')

In [ ]:
HUC8_json

In [ ]:
from folium.plugins import MousePosition, Search

m_chla = folium.Map(location=[48, -102], zoom_start=3)

bins = [0,2,7,30,HUC8_NARS_NLA_FINAL_chla['CHLX_RESULT'].max()]

CHLX_RESULT_layer = folium.Choropleth(
    geo_data=HUC8_json,
    name='Chlorophyll-a',
    data=HUC8_NARS_NLA_FINAL_chla,
    columns=['HUC_8', 'CHLX_RESULT'],
    key_on='feature.properties.HUC_8',
    bins = bins,
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='grey',
    nan_fill_opacity=0.7,
    legend_name=r"Chlorophyll-a (microg/L)",
    highlight=True,
).add_to(m_chla)

folium.GeoJson(
    HUC8_NARS_NLA_FINAL_chla.to_json(),
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.1,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'CHLX_RESULT'],
        aliases=['HUC8 watershed', 'Chlorophyll-a (microg/L)'], 
        localize=True)).add_to(m_chla)

folium.LayerControl().add_to(m_chla)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_chla)

HUC8search = Search(
    layer=CHLX_RESULT_layer,
    geom_type='Polygon',
    placeholder='Search for a HUC8 watershed',
    collapsed=False,
    search_label='HUC_8',
    weight=3
).add_to(m_chla)

m_chla.save('HUC8_chla.html')